In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import json

model = ChatOllama(
    model="llama3.1",
)

In [ ]:
user_prompt = "Teach me about machine learning in R"

# Generate Initial Course Data


In [ ]:
base_course_prompt = """You are a content creation wizard. Given a user query about a course they would like to learn about, 
generate a title, description, and a comma-separated list of 5-10 logical sections 
or modules that would make up a comprehensive curriculum for that course in JSON format. 
The output should be in JSON format with the fields title, description, and modules. 
Ensure the modules flow in a sensible order from beginner to more advanced topics. 
The modules should appear as natural language. 
Title and description fields will be a normal string and the module will be a list of strings. 
Remember, do not say anything else, just the JSON object."""

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

base_course_examples = [
    {
        "input": "Teach me about making games in C",
        "output": """{
  "title": "Game Development with C: From Basics to Advanced Techniques",
  "description": "Learn the fundamentals and best practices of creating engaging games using the C programming language. This comprehensive course covers game development basics, game loop, event handling, graphics, sound, AI, physics, and more.",
  "modules": [
    "Introduction to Game Development with C: Setting up a Development Environment",
    "Game Loop Fundamentals: Understanding the Main Loop, Input Handling, and Timing",
    "Event Handling in Games: Mouse, Keyboard, and Gamepad Inputs",
    "2D Graphics Programming with SDL and OpenGL: Drawing Shapes, Textures, and Sprites",
    "Sound Design for Games: Creating Audio Effects and Music with FMOD and OpenAL",
    "Game AI and Pathfinding: Implementing Simple AI Behaviors and Complex Routing Algorithms",
    "Physics Engines in Games: Using Bullet Physics and PhysX to Simulate Real-World Dynamics",
    "Advanced Game Development Topics: Networking, Multiplayer, and Optimization Techniques",
    "Project Development: Creating a Complete 2D Game with C, SDL, OpenGL, and FMOD"
  ]
}""",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

base_course_few_shot_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, examples=base_course_examples
)

base_course_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", base_course_prompt),
        base_course_few_shot_template,
        ("human", "{input}"),
    ]
)

base_course_chain = base_course_prompt | model

In [ ]:
welcome_data_str = ""
for chunk in base_course_chain.stream(
    {
        "input": user_prompt,
    }
):
    welcome_data_str += chunk.content
    print(chunk.content, end="", flush=True)

In [ ]:
course_data = json.loads(welcome_data_str)

In [ ]:
print(course_data["title"])
print(course_data["description"])
print(course_data["modules"])

# Generate modules prompt


In [ ]:
section_prompt = """You are an expert curriculum designer and educational content creator. 
Your task is to create a detailed content structure for a course module. Please follow these guidelines: 
Start each module with a short introduction of the topic. Follow that with walking the student through 
the 5-10 key concepts needed to understand the topic in great detail. The concepts should be descriptive and robust. 
If possible, provide examples to help reinforce concepts. 
End the module with at least 3 practice problems to understand the topic ranging from easy, medium, to hard. 
Remember not to cover a topic that was previously covered in another module. 
Ensure your output is in Markdown format and do not say anything else."""

In [ ]:
section_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            section_prompt,
        ),
        (
            "human",
            "{input}",
        ),
    ]
)

section_chain = section_prompt_template | model

In [ ]:
documents = []
for i, module in enumerate(course_data["modules"]):
    document = {"title": module, "content": ""}
    for chunk in section_chain.stream(
        {
            "input": f"""Course Title: { course_data["title"]}\nCourse Description: { course_data["description"]}\nPreviously covered modules: {module[i:]}\nModule Title: {module}\nModule Number:{i+1}"""
        }
    ):
        document["content"] += chunk.content
        print(chunk.content, end="", flush=True)
    documents.append(document)

In [ ]:
from IPython.display import display, Markdown

for document in documents:
    display(Markdown(document["content"]))